# Nifty - 50 stock price prediction

## Input & Training

### Scrape the daily data of all stocks in NIFTY - 50 for the last 4 years.

In [1]:
# importing required modules to collect stock details

from datetime import date
import datetime
from jugaad_data.nse import bhavcopy_save, bhavcopy_fo_save

from jugaad_data.nse import stock_df

# importing required modules to analyse and connect google sheet

import pandas as pd
import gspread
import pandasql as ps
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sb
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn import svm
from xgboost import XGBClassifier


### Taking input from user

#### Connecting with google sheet

In [2]:
gc = gspread.service_account(filename='nifty50-stock-price-prediction-39536132386e.json')
Users_Sheet = gc.open("Users_Sheet")



from google.oauth2.service_account import Credentials
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]
credentials = Credentials.from_service_account_file(
    'nifty50-stock-price-prediction-39536132386e.json',
    scopes=scopes
)
gc = gspread.authorize(credentials)

In [3]:
# to update in google sheet

f_r = gc.open("Users_Sheet").sheet1

### Calculating earned profit from historical data

In [4]:
# making dataframe of users inputed data

user_df = pd.DataFrame(Users_Sheet.sheet1.get('A1:D2'))

In [5]:
# converting first row to header

new_header = user_df.iloc[0]
user_df = user_df[1:]
user_df.columns = new_header

In [6]:
# changing to suitable datatypes

user_df['Buying Date'] =  pd.to_datetime(user_df['Buying Date'])
user_df['Selling Date'] =  pd.to_datetime(user_df['Selling Date'])
user_df['Quantity'] =  pd.to_numeric(user_df['Quantity'])

In [7]:
# checking the formates of inputed data

print(user_df)
user_df.dtypes

0 Stock Name  Quantity Buying Date Selling Date
1  COALINDIA      1000  2022-10-27   2023-10-28


0
Stock Name              object
Quantity                 int64
Buying Date     datetime64[ns]
Selling Date    datetime64[ns]
dtype: object

In [8]:
# collecting single data values from user input dataframe

Stock_Name = user_df['Stock Name'].iloc[0]
Quantity = user_df['Quantity'].iloc[0]
Buying_Date = user_df['Buying Date'].iloc[0]
Selling_Date = user_df['Selling Date'].iloc[0]
Buying_Date = str(Buying_Date)[:10]
Selling_Date = str(Selling_Date)[:10]

In [9]:
# checking the collected data from user

print(Stock_Name,Quantity,Buying_Date,Selling_Date)

COALINDIA 1000 2022-10-27 2023-10-28


In [10]:
# collecting stock dataframe of given stock symbol

# df = stock_df(symbol=Stock_Name, from_date=date(2019,1,1),to_date=date(2022,10,31), series="EQ")
df = stock_df(symbol=Stock_Name, from_date=date(2019,1,1),to_date=date.today(), series="EQ")

In [11]:
# changing to suitable data type

df = df.astype({'DATE':'string'})

In [12]:
# taking average price of a day to calculate profit

df['Day Price'] = (df['OPEN']+df['CLOSE'])/2

### Train a stock price prediction model for each stock.

In [13]:
df2 = df[['DATE','OPEN','HIGH','LOW','CLOSE','VOLUME','Day Price']]
df2.head(2)

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,Day Price
0,2022-11-04,246.7,248.7,245.5,246.05,5511411,246.375
1,2022-11-03,244.0,247.6,241.0,246.20,6754654,245.100


In [14]:
splitted = df2['DATE'].str.split('-', expand=True)
 
df2['MONTH'] = splitted[1].astype('int')
df2['YEAR'] = splitted[0].astype('int')
df2['DAY'] = splitted[2].astype('int')
 
df2.head()

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,Day Price,MONTH,YEAR,DAY
0,2022-11-04,246.70,248.7,245.50,246.05,5511411,246.375,11,2022,4
1,2022-11-03,244.00,247.6,241.00,246.20,6754654,245.100,11,2022,3
2,2022-11-02,245.65,249.6,242.15,245.80,14974441,245.725,11,2022,2
3,2022-11-01,247.00,247.0,243.25,245.65,8813106,246.325,11,2022,1
4,2022-10-31,246.00,247.5,243.55,245.95,8365329,245.975,10,2022,31


In [15]:
df2 = df2.drop(['DATE','OPEN','HIGH','LOW','CLOSE','VOLUME'],axis=1)
df2.head()

,Day Price,MONTH,YEAR,DAY
0,246.375,11,2022,4
1,245.100,11,2022,3
2,245.725,11,2022,2
3,246.325,11,2022,1
4,245.975,10,2022,31


### Extract the independent and dependent variables

In [16]:
X=df2.iloc[:,1:].values
Y=df2.iloc[:,-4].values

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.3,random_state=4)

In [19]:
reg.fit(X_train, Y_train)

LinearRegression()

In [20]:
features = np.array([[int(Buying_Date[5:7]),int(Buying_Date[:4]),int(Buying_Date[-2:])]])
PBP = reg.predict(features)
features = np.array([[int(Selling_Date[5:7]),int(Selling_Date[:4]),int(Selling_Date[-2:])]])
PSP = reg.predict(features)

### Evaluate the model using MSE, RMSE and R square

In [21]:
Y_pred=reg.predict(X_test)

In [22]:
MSE=metrics.mean_squared_error(Y_test, Y_pred)

In [23]:
RMSE=np.sqrt(MSE)
RMSE

43.74997237386539

In [24]:
R2=metrics.r2_score(Y_test,Y_pred)
R2

0.05199445731805985

In [25]:
if user_df['Buying Date'].iloc[0] < date.today():
    Buy_price = df[df['DATE']==Buying_Date]['Day Price'].iloc[0]
else:
    Buy_price = PBP[0]
    
if user_df['Selling Date'].iloc[0] < date.today():
    Sale_price = df[df['DATE']==Selling_Date]['Day Price'].iloc[0]
else:
    Sale_price = PSP[0]

In [26]:
# updating the buying price, selling price and profit to google sheet

f_r.update_cell(1,9,Buy_price)
f_r.update_cell(2,9,Sale_price)
f_r.update_cell(3,9,PBP[0])
f_r.update_cell(4,9,PSP[0])

{'spreadsheetId': '1PWgtWZH8Ln6Ed3XHrCROlStmmDm_0OdwiUjqnNhAGA4',
 'updatedRange': 'Sheet1!I4',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}